In [1]:
import numpy as np
class Perceptron:
    def __init__(self, weights, desired_output, bias, eta):  
        self.weights = np.array(weights)
        self.bias = bias
        self.desired_output = desired_output
        self.eta = eta
        self.activity = 0.0
        self.activation = 0.0
        self.delta = 0.0

    # Calculate activity
    def calc_activity(self, input):
        input = np.array(input)
        self.activity = self.bias + np.dot(self.weights, input)
    
    # Calculate activation with sigmoid function
    def calc_activation(self):
        self.activation = 1 / (1 + np.exp(-self.activity))
    
    # Calculate delta/error
    def set_delta(self, error):
        activation_derivative = self.activation * (1 - self.activation)
        self.delta = error * activation_derivative

    # Update weights of each perceptron
    def update_weights(self, inputs):
        self.weights -= self.eta * self.delta * np.array(inputs)
        self.bias -= self.eta * self.delta

In [2]:
class Network:
    def __init__(self, input_size=2, hidden_size=2, output_size=1, eta=1.0):
        self.eta = eta

        # Hidden layer perceptrons
        self.hidden_layer = [Perceptron(weights=[0.3] * input_size, desired_output=0, bias=0, eta=eta)
                             for _ in range(hidden_size)]
        
        # Output layer perceptrons
        self.output_layer = [Perceptron(weights=[0.8] * hidden_size, desired_output=0.7, bias=0, eta=eta)]

    # Feed-Forward
    def forward(self, inputs):
        # Hidden layer feed forward
        hidden_activations = []
        for perceptron in self.hidden_layer:
            perceptron.calc_activity(inputs)
            perceptron.calc_activation()
            hidden_activations.append(perceptron.activation)
        
        # Output layer feed forward
        for perceptron in self.output_layer:
            perceptron.calc_activity(hidden_activations)
            perceptron.calc_activation()
        
        return hidden_activations, self.output_layer[0].activation

    def ffbp(self, inputs, desired_output):
        # Feed Forward
        hidden_activations, output_activation = self.forward(inputs)

        # Get output layer error and delta
        output_error = output_activation - desired_output
        self.output_layer[0].set_delta(output_error)

        # Back propagate to hidden layer
        for i, hidden_neuron in enumerate(self.hidden_layer):
            hidden_error = self.output_layer[0].weights[i] * self.output_layer[0].delta
            hidden_neuron.set_delta(hidden_error)

        # Update weights for output layer
        self.output_layer[0].update_weights(hidden_activations)

        # Update weights for hidden layer
        for i, hidden_neuron in enumerate(self.hidden_layer):
            hidden_neuron.update_weights(inputs)

    # Train Network for certain number of epochs
    def train(self, inputs, desired_output, epochs):
        for _ in range(epochs):
            for i in range(len(inputs)):
                self.ffbp(inputs[i], desired_output[i])

    # Plug in an input to the trained network
    def predict(self, inputs):
        _, output_activation = self.forward(inputs)
        return output_activation
    
    # Get error
    def getBigE(self, inputs, desired_output):
        output_activation = self.predict(inputs)
        small_e = desired_output - output_activation
        bigE = 0.50 * small_e ** 2
        return bigE
    
    # Get weights
    def getWeights(self):
        weights_info = {}
        
        # Get weights and biases from hidden layer perceptrons
        for i, perceptron in enumerate(self.hidden_layer):
            weights_info[f"Hidden Layer Perceptron {i + 1}"] = {
                "weights": perceptron.weights,
                "bias": perceptron.bias
            }
        
        # Collect weights and biases from output layer perceptron
        for i, perceptron in enumerate(self.output_layer):
            weights_info["Output Layer Perceptron"] = {
                "weights": perceptron.weights,
                "bias": perceptron.bias
            }
        return weights_info

In [3]:
# TESTING THE FFBP MULTI-Layer NEURAL NETWORK
# Single input set [1, 2] with a desired output of 0.7
inputs = np.array([[1, 2]])
desired_output = np.array([0.7])

# Initialize the neural network
neural_network = Network(input_size=2, hidden_size=2, output_size=1, eta=1.0)

# Train the network
neural_network.train(inputs, desired_output, epochs=500)

# Test the network
print("Final output after training:")
for i in range(len(inputs)):
    print(f"Input: {inputs[i]}, Predicted Output: {neural_network.predict(inputs[i])}")

Final output after training:
Input: [1 2], Predicted Output: 0.7


In [4]:
# Method 1. Two input pairs training, looped 15 times.
inputs = np.array([[1, 1], [-1, -1]])  # Two inputs, each is a 1D array
desired_output = np.array([0.9, 0.05])  # Target outputs

# Initialize the neural network
neural_network = Network(input_size=2, hidden_size=2, output_size=1, eta=1.0)

# Training loop
for _ in range(15):
    neural_network.train([inputs[0]], [desired_output[0]], epochs=1)
    neural_network.train([inputs[1]], [desired_output[1]], epochs=1)

# Retrieve and print updated weights
weights_info = neural_network.getWeights()
print(f"Updated weights after training: {weights_info}")

# Print final output after training
print("Final output after training:")
print("Output for (1,1)", neural_network.predict(inputs[0]))  # This prints without concatenation issues
print("Big E:", neural_network.getBigE(inputs[0], desired_output[0]))
print("Output for (-1,-1), ", neural_network.predict(inputs[1]))
print("Big E:", neural_network.getBigE(inputs[1], desired_output[1]))


Updated weights after training: {'Hidden Layer Perceptron 1': {'weights': array([0.69205177, 0.69205177]), 'bias': np.float64(-0.12850040822123737)}, 'Hidden Layer Perceptron 2': {'weights': array([0.69205177, 0.69205177]), 'bias': np.float64(-0.12850040822123737)}, 'Output Layer Perceptron': {'weights': array([0.95185797, 0.95185797]), 'bias': np.float64(-0.8257813486188029)}}
Final output after training:
Output for (1,1) 0.6583208713508027
Big E: 0.029204400612317622
Output for (-1,-1),  0.3817659623378531
Big E: 0.055034326882980884


In [5]:
# Method 2. Looped training of input 1 15 times and subsequent looped training of input 2 15 times.
inputs = np.array([[1, 1], [-1, -1]])  # Two inputs, each is a 1D array
desired_output = np.array([0.9, 0.05])  # Target outputs

# Initialize the neural network
neural_network = Network(input_size=2, hidden_size=2, output_size=1, eta=1.0)

# Training loop
neural_network.train([inputs[0]], [desired_output[0]], epochs=15)
neural_network.train([inputs[1]], [desired_output[1]], epochs=15)

# Retrieve and print updated weights
weights_info = neural_network.getWeights()
print(f"Updated weights after training: {weights_info}")

# Print final output after training
print("Final output after training:")
print("Output for (1,1)", neural_network.predict(inputs[0]))
print("Big E:", neural_network.getBigE(inputs[0], desired_output[0]))
print("Output for (-1,-1), ", neural_network.predict(inputs[1]))
print("Big E:", neural_network.getBigE(inputs[1], desired_output[1]))

Updated weights after training: {'Hidden Layer Perceptron 1': {'weights': array([0.57602653, 0.57602653]), 'bias': np.float64(-0.17015279256338045)}, 'Hidden Layer Perceptron 2': {'weights': array([0.57602653, 0.57602653]), 'bias': np.float64(-0.17015279256338045)}, 'Output Layer Perceptron': {'weights': array([0.58941714, 0.58941714]), 'bias': np.float64(-1.1735567184480915)}}
Final output after training:
Output for (1,1) 0.4216576338099176
Big E: 0.11440570964616345
Output for (-1,-1),  0.2838448109486975
Big E: 0.027341697803816043
